
![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/public/NER_AR.ipynb)

# **Recognize entities in Arabic text**

# **Colab Setup**

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

# Install Spark NLP Display lib
! pip install --upgrade -q spark-nlp-display

# **Start the Spark session**

In [2]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

import pandas as pd

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

import string
import numpy as np


spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

Spark NLP version 4.2.8
Apache Spark version: 3.3.0


# **🔎 For about model**


📌 **"aner_cc_300d"**--> *This model uses Arabic word embeddings to find 4 different types of entities in Arabic text. It is trained using **arabic_w2v_cc_300d** word embeddings, so please use the same embeddings in the pipeline.*

*   Predicted Entities => **Persons-PER, Locations-LOC, Organizations-ORG, Miscellaneous-MISC.**





# **🔎Define Spark NLP pipeline**

In [3]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained("arabic_w2v_cc_300d", "ar") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings")

ner = NerDLModel.pretrained("aner_cc_300d", "ar" ) \
    .setInputCols(["document", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["document", "token", "ner"]) \
    .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(
    stages = [
        documentAssembler, 
        tokenizer,
        embeddings,
        ner,
        ner_converter])

empty_df = spark.createDataFrame([['']]).toDF("text")
pipelineModel = nlpPipeline.fit(empty_df)
light_model = LightPipeline(pipelineModel)

arabic_w2v_cc_300d download started this may take some time.
Approximate size to download 1.1 GB
[OK!]
aner_cc_300d download started this may take some time.
Approximate size to download 14.1 MB
[OK!]


# **🔎Sample text**

In [4]:
text = """في الخامس من نوفمبر ألتقى ريال مدريد مع فريق راؤول السابق أتلتكو مدريد ومن المصادفة بأن أول أهداف راؤول في الليغا كانت في مرمى أتلتي , وفي نفس المباراة ساهم راؤول في ركلة جزاء نفذها المهاجم التشبلي زامورنوا , وفي السابع من يناير لعام 1995 دخل راؤول بديلا في دقيقة 64 في مباراة برشلونة وكان أول كلاسيكو للفتى الذهبي , وفي 22 من يناير سجل راؤول هدفين في مرمى سلتا فيغو , وأستطاع اللاعب البالغ من العمر 17 فقط أن ينهى الموسم ب تسجيل 9 أهداف في الليغا الأسبانية , وحاز فزيقه على لقب الدوري الأسباني , وحقق جائزة دون بول لأفضل لاعب شاب في موسم 1994 - 95 , ولعب راؤول 28 مباراة في الدوري وسجل 9 أهداف وصنع أربعة أهداف , بينما في كأس إسبانيا لعب مباراتين وسجل هدف , وساهم بروز راؤول في ذلك الموسم إلى إستنغاءعن الصقر الأسباني اميليو بوتراغوينيو , بالإضافة للرحيل ألفونسو بيريز إلى ريال بيتس ."""

# **🔎Run the pipeline**

In [5]:
light_result = light_model.fullAnnotate(text)

chunks = []
entities = []
begin =[]
end = []

for n in light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    
import pandas as pd

result_df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                                   'begin': begin, 'end': end})
result_df

Before _validateStagesInputCols


,chunks,entities,begin,end
0,ريال مدريد,ORG,26,35
1,راؤول,PER,45,49
2,أتلتكو مدريد,ORG,58,69
3,راؤول,PER,98,102
4,الليغا,MIS,107,112
5,راؤول,PER,157,161
6,زامورنوا,PER,198,205
7,راؤول,PER,243,247
8,برشلونة,LOC,277,283
9,راؤول,PER,337,341


# **🔎Visualize results**

In [6]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = light_result[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)